In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [4]:
taxi = sc.textFile('yellow.csv.gz')
bike = sc.textFile('citibike.csv')

In [7]:
list(enumerate(bike.first().split(',')))

[(0, 'cartodb_id'),
 (1, 'the_geom'),
 (2, 'tripduration'),
 (3, 'starttime'),
 (4, 'stoptime'),
 (5, 'start_station_id'),
 (6, 'start_station_name'),
 (7, 'start_station_latitude'),
 (8, 'start_station_longitude'),
 (9, 'end_station_id'),
 (10, 'end_station_name'),
 (11, 'end_station_latitude'),
 (12, 'end_station_longitude'),
 (13, 'bikeid'),
 (14, 'usertype'),
 (15, 'birth_year'),
 (16, 'gender')]

In [22]:
#bike RDD -> filter by greenwich ave & 8 ave station 
def bikeFilter(records):
    for record in records: 
        fields = record.split(',')
        if (fields[6] =='Greenwich Ave & 8 Ave' and
                fields[3].startswith('2015-02-01')):
            yield fields[3]

matchedBike = bike.mapPartitions(bikeFilter)

In [23]:
matchedBike.take(2)

['2015-02-01 00:05:00+00', '2015-02-01 00:05:00+00']

In [37]:
def filterTaxi(pIndex, lines):
    if pIndex==0:
        next(lines)
    import pyproj
    proj = pyproj.Proj(init="epsg:2263", preserve_units=True)
    station = proj(-74.00263761, 40.73901691)
    radius = 1320**2
    for trip in lines:
        fields = trip.split(',')
        try: 
            dropoff = proj(fields[5], fields[4])
        except:
            continue
        if(fields[1].startswith('2015-02-01') and 
           ((dropoff[0] - station[0])**2 + 
            (dropoff[1] - station[1])**2 <=radius)):
            yield dropoff
        
matchedTaxi = taxi.mapPartitionsWithIndex(filterTaxi)
matchedTaxi.count()

7278